In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import qualified Data.Attoparsec.Char8 as A
import Data.ByteString.UTF8 (toString)
import Data.List (foldl', nub)
import qualified Data.Map.Strict as M

input <- dayLines 8

In [2]:
data Instr = Instr
    { instrRegister  :: String
    , instrOperation :: Int
    , instrAmount    :: Int
    , instrCondition :: Cond
    }

data Cond = Cond
    { condRegister   :: String
    , condComparison :: (Int -> Int -> Bool)
    , condValue      :: Int
    }

In [3]:
notSpace = (/= ' ')

convertComparison c = case c of
    "<"  -> (<)
    "<=" -> (<=)
    "==" -> (==)
    ">=" -> (>=)
    ">"  -> (>)
    "!=" -> (/=)

parseCond = do
    register <- toString <$> A.takeWhile notSpace
    A.space
    comp <- convertComparison <$> A.takeWhile notSpace
    A.space
    n <- A.signed A.decimal
    return $ Cond register comp n

parseInstr = do
    register <- toString <$> A.takeWhile notSpace
    A.space
    op <- A.choice $ map A.string ["inc", "dec"]
    let operation = if op == "inc" then 1 else -1
    A.space
    number <- A.signed A.decimal
    A.string " if "
    cond <- parseCond
    return $ Instr register operation number cond


parsedInstructions =  map (either error id . A.parseOnly parseInstr) input

In [4]:
registers = nub $ concatMap (\i -> [instrRegister i, condRegister (instrCondition i)]) parsedInstructions

initialRegisters = M.fromList $ zip registers $ repeat 0

In [5]:
evaluateCondition :: M.Map String Int -> Cond -> Bool
evaluateCondition registers condition = let
    registerValue = registers M.! condRegister condition
    compValue     = condValue condition
    in (condComparison condition) registerValue compValue

In [6]:
go :: M.Map String Int -> Instr -> M.Map String Int
go state current = if evaluateCondition state (instrCondition current)
    then M.adjust (+(instrOperation current * instrAmount current)) (instrRegister current) state
    else state

finished = foldl' go initialRegisters parsedInstructions

maximum $ M.elems finished

5215

In [7]:
states = concatMap M.elems $ scanl go initialRegisters parsedInstructions
maximum states

6419